In [11]:
run ~/w/replab/replab_init

# Separability threshold of qubit-qubit Werner states

Those states are invariant under a joint rotation by qubit unitary.

So we declare the symmetry group: $U(2)$ identically acting on each subsystem.

We do not use declare the symmetry of subsystem exchange as it would complicate the PPT constraint.

In [16]:
G = replab.U(2);
% The representation of the two qubit state
rep = kron(G.definingRep, G.definingRep);
% The representation acting on the partially transposed state
repT = kron(G.definingRep, dual(G.definingRep));

In [23]:
% Spaces of equivariant Hermitian matrices
H = rep.hermitianInvariant;
HT = repT.hermitianInvariant;

## The partial transpose as an equivariant map

In [29]:
% The partial transpose linear map
pptFun = @(X) reshape(permute(reshape(X, [2 2 2 2]), [3 2 1 4]), [4 4]);
% Upgraded as an equivariant super operator from the space H to the space HT
ppt = replab.equiop.generic(H, HT, pptFun);
ppt.check

Checking equivariant space map...
Checking map is equivariant...


## Optimization problem

In [35]:
% Define the singlet state as an equivar
singlet = replab.equivar(H, 'value', [0 0 0 0; 0 1 -1 0; 0 -1 1 0; 0 0 0 0]/2);
% Define the noise as an equivar
noise = replab.equivar(H, 'value', eye(4)/4);

In [36]:
% Standard sdpvar
t = sdpvar;
% equivar*sdpvar works, but not sdpvar*equivar (as sdpvar would provide the * op and doesn't handle equivar)
rho = singlet*t + noise*(1-t);

We use the syntax `sdp(EV)` to define a semidefinite positive constraint, where EV is an equivar.
We use the syntax `value(EV)` to recover the sdpvar in the original (i.e. non symmetry adapted) basis.

In [39]:
C = [trace(value(rho)) == 1
     issdp(rho)
     issdp(ppt(rho))]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
|   ID|                    Constraint|          Coefficient range|
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
|   #1|       Equality constraint 1x1|   2.2204e-16 to 3.3307e-16|
|   #2|   Element-wise inequality 1x1|               0.25 to 0.75|
|   #3|   Element-wise inequality 1x1|               0.25 to 0.25|
|   #4|   Element-wise inequality 1x1|               0.25 to 0.75|
|   #5|   Element-wise inequality 1x1|               0.25 to 0.25|
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [40]:
% note that this is a linear program now
optimize(C, -t)


Optimal solution found.

ans = 
  struct with fields:

    yalmipversion: '20200930'
    matlabversion: '9.7.0.1190202 (R2019b)'
       yalmiptime: 0.1537
       solvertime: 0.0578
             info: 'Successfully solved (LINPROG)'
          problem: 0


In [41]:
value(t)

ans =
    0.3333
